This file creates embeddings using gemini models

References:

https://docs.trychroma.com/docs/overview/introduction 
https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-multimodal-embeddings#python 

https://github.com/google-gemini/cookbook/blob/main/examples/chromadb/Vectordb_with_chroma.ipynb 

https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro_multimodal_embeddings.ipynb#scrollTo=RQT500QqVPIb 


In [2]:
%pip install google-cloud-aiplatform
%pip install chromadb


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ----------------------------- ---------- 5.8/7.8 MB 39.6 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 40.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 31.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl.metadata (592 bytes)
Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl (434 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.73.1 requires protobuf<7.0.0,>=6.30.0, but you have protobuf 5.29.5 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# Example code to store a single image

"""

import vertexai
from vertexai.vision_models import Image, MultiModalEmbeddingModel

# TODO: Replace with your actual Google Cloud project ID
PROJECT_ID = "verse-dev-433901"
vertexai.init(project=PROJECT_ID, location="us-central1")

embedding_dimension = 128

model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

# Load image from local 'frames' folder
image = Image.load_from_file("frames/frame_15.jpg")

# Optional: Add contextual text if you want to provide additional info
contextual_text = "Describe the image content here or leave empty"

embeddings = model.get_embeddings(
    image=image,
    contextual_text=contextual_text,
    dimension=embedding_dimension,
)

print(f"Image Embedding: {embeddings.image_embedding}")
print(f"Text Embedding: {embeddings.text_embedding}")

"""

'\n\nimport vertexai\nfrom vertexai.vision_models import Image, MultiModalEmbeddingModel\n\n# TODO: Replace with your actual Google Cloud project ID\nPROJECT_ID = "verse-dev-433901"\nvertexai.init(project=PROJECT_ID, location="us-central1")\n\nembedding_dimension = 128\n\nmodel = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")\n\n# Load image from local \'frames\' folder\nimage = Image.load_from_file("frames/frame_15.jpg")\n\n# Optional: Add contextual text if you want to provide additional info\ncontextual_text = "Describe the image content here or leave empty"\n\nembeddings = model.get_embeddings(\n    image=image,\n    contextual_text=contextual_text,\n    dimension=embedding_dimension,\n)\n\nprint(f"Image Embedding: {embeddings.image_embedding}")\nprint(f"Text Embedding: {embeddings.text_embedding}")\n\n'

In [5]:
import os
import vertexai
from vertexai.vision_models import Image, MultiModalEmbeddingModel
import chromadb

# Initialize Vertex AI
PROJECT_ID = "verse-dev-433901"
vertexai.init(project=PROJECT_ID, location="us-central1")

# Load Gemini multimodal embedding model
embedding_dimension = 128
model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

# Initialize Chroma PersistentClient to save/load DB locally
chroma_client = chromadb.PersistentClient(path="chromadb_store")

# Create or get collection; embedding_function=None because we pass embeddings manually
collection = chroma_client.get_or_create_collection(
    name="video_image_embeddings",
    embedding_function=None
)

# Load top 10 images sorted by filename
frames_folder = "frames"
image_files = sorted([f for f in os.listdir(frames_folder) if f.endswith(('.jpg', '.png'))])[:10]

for i, filename in enumerate(image_files):
    image_path = os.path.join(frames_folder, filename)
    image = Image.load_from_file(image_path)

    # Get image embeddings (no contextual text)
    embeddings = model.get_embeddings(
        image=image,
        dimension=embedding_dimension
    )

    vector = embeddings.image_embedding

    # Add to Chroma collection
    collection.add(
        ids=[f"img_{i}"],
        documents=[f"Frame from {filename}"],
        embeddings=[vector],
        metadatas=[{"filename": filename}]
    )

    print(f"\nEmbedding for {filename} (length {len(vector)}):\n{vector[:5]} ...")  # Print first 5 dims

# Optional: Use heartbeat or reset if needed
# print(chroma_client.heartbeat())
# chroma_client.reset()  # Be careful: this deletes all data


C:\Users\tm0792.STUDENTS.007\AppData\Roaming\Python\Python313\site-packages\vertexai\_model_garden\_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
C:\Users\tm0792.STUDENTS.007\AppData\Roaming\Python\Python313\site-packages\vertexai\vision_models\_vision_models.py:153: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
Failed to send telemetry event CollectionAd


Embedding for frame_0.jpg (length 128):
[0.031740237, 0.0485489257, -0.0125791235, -0.0115507226, -0.441018879] ...

Embedding for frame_1.jpg (length 128):
[0.0753487349, 0.00619010394, -0.051838439, 0.017776316, -0.456335813] ...

Embedding for frame_10.jpg (length 128):
[0.0765796378, 0.0287108198, -0.0599589683, -0.0164515115, -0.455635041] ...

Embedding for frame_11.jpg (length 128):
[0.040443249, 0.0347772874, -0.0184420217, 0.0105499672, -0.421787679] ...

Embedding for frame_12.jpg (length 128):
[0.0518487431, 0.00713418564, 0.000910361705, -0.0317022838, -0.443772018] ...

Embedding for frame_13.jpg (length 128):
[0.0849476531, 0.023952473, -0.0130670303, -0.00410143426, -0.427604258] ...

Embedding for frame_14.jpg (length 128):
[0.00785079878, 0.0747391507, -0.0248102937, -0.0493642837, -0.407607168] ...

Embedding for frame_15.jpg (length 128):
[0.126422316, 0.0398501642, -0.0688068643, -0.0121922446, -0.459890515] ...

Embedding for frame_16.jpg (length 128):
[0.04278413